In [6]:
import cv2
import numpy as np

In [7]:

template = cv2.imread('IMG_0179.jpg', 0)

In [8]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Get template dimensions
h, w = template.shape[:2]

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

canny_threshold1 = 100
canny_threshold2 = 250

prev_top_left = None
prev_bottom_right = None
tracking_enabled = False

[ WARN:0@3407.134] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@3408.217] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements


In [9]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    # Apply Canny edge detection
    edges = cv2.Canny(gray_frame, canny_threshold1, canny_threshold2)
    
    if tracking_enabled:
        # Define search area based on previous detection
        search_area_top_left = (max(0, prev_top_left[0] - w), max(0, prev_top_left[1] - h))
        search_area_bottom_right = (min(frame.shape[1], prev_bottom_right[0] + w), min(frame.shape[0], prev_bottom_right[1] + h))
        search_area = gray_frame[search_area_top_left[1]:search_area_bottom_right[1], search_area_top_left[0]:search_area_bottom_right[0]]
        
        # Ensure search area size is compatible with template
        if search_area.shape[0] < h or search_area.shape[1] < w:
            tracking_enabled = False
        else:
            # Apply cross correlation in the search area
            result = cv2.matchTemplate(search_area, template, cv2.TM_CCOEFF)
            
            # Find maximum correlation value and its location    
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            max_loc = (max_loc[0] + search_area_top_left[0], max_loc[1] + search_area_top_left[1])
            
            # Draw bounding box around the detected object
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
            prev_top_left = top_left
            prev_bottom_right = bottom_right
    else:
        # Apply cross correlation on the entire frame
        result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF)
        
        _, max_val, _, max_loc = cv2.minMaxLoc(result)    
        threshold = 2.0
        
        if max_val > threshold:

            tracking_enabled = True
            prev_top_left = max_loc
            prev_bottom_right = (prev_top_left[0] + w, prev_top_left[1] + h)
            
            # Draw bounding box around the detected object
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
    out.write(frame)
    cv2.imshow('Face Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

: 